# Import Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Dropout, Lambda, Flatten
from keras.optimizers import Adam ,RMSprop
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.preprocessing.image import ImageDataGenerator

# Load data

In [ ]:
train = pd.read_csv(r"../input/Kannada-MNIST/train.csv")
print(train.shape)
train.head()

In [ ]:
test = pd.read_csv(r"../input/Kannada-MNIST/test.csv")
print(test.shape)
test.head()

In [ ]:
X_train = (train.iloc[:,1:].values).astype('float32')
y_train = train.iloc[:,0].values.astype('int32')
X_test = (test.iloc[:,1:].values).astype('float32')

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

# Data Visulaization

In [ ]:
X_train = X_train.reshape(X_train.shape[0],28,28)

for i in range(1,4):
    plt.subplot(329+(i+1))
    plt.imshow(X_train[i],cmap=plt.get_cmap('gray'))
    plt.title(y_train[i])

In [ ]:
X_train = X_train.reshape(X_train.shape[0],28,28,1) 
X_test = X_test.reshape(X_test.shape[0],28,28,1) 
print(X_train.shape) 
print(X_test.shape)

# Preprocessing data images

# Feature Standardization

In [ ]:
meanpx = X_train.mean().astype(np.float32)
stdpx = X_train.std().astype(np.float32)

def standardize(x):
    return (x-meanpx)/stdpx

# One-hot encoding of labels

In [ ]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
num_classes = y_train.shape[1]
num_classes

In [ ]:
plt.title(y_train[7])
plt.plot(y_train[7])
plt.xticks(range(10));

# Designing Neural Net Architecture

# Linear Model

In [ ]:
from keras.models import Sequential 
from keras.layers.core import Lambda, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization, Convolution2D , MaxPooling2D

In [ ]:
model = Sequential()
model.add(Lambda(standardize,input_shape=(28,28,1)))
model.add(Flatten())
model.add(Dense(10,activation='softmax'))

In [ ]:
from keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.preprocessing import image
gen = image.ImageDataGenerator()

In [ ]:
from sklearn.model_selection import train_test_split
X = X_train
y = y_train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)
batches = gen.flow(X_train, y_train, batch_size=64)
val_batches=gen.flow(X_val, y_val, batch_size=64)

In [ ]:
history=model.fit_generator(generator=batches, steps_per_epoch=X_train.shape[0]//64, epochs=30, 
                    validation_data=val_batches, validation_steps=val_batches.n)

In [ ]:
model.evaluate(X_train,y_train) #train loss and train accuracy

# Fully Connected Layer

In [ ]:
def fully_connected_layer():
    model=Sequential([
        Lambda(standardize,input_shape=(28,28,1)),
        Flatten(),
        Dense(512,activation='relu'),
        Dense(10,activation='softmax')
    ])
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
fc = fully_connected_layer()
fc.optimizer.lr=0.01

In [ ]:
history=model.fit_generator(generator=batches, steps_per_epoch=X_train.shape[0]//64, epochs=10, 
                    validation_data=val_batches, validation_steps=val_batches.n)

# Convolution Neural Network

In [ ]:
from keras.layers import Convolution2D, MaxPooling2D

def get_cnn_model():
    model = Sequential([
        Lambda(standardize, input_shape=(28,28,1)),
        Convolution2D(32,(3,3), activation='relu'),
        Convolution2D(32,(3,3), activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,(3,3), activation='relu'),
        Convolution2D(64,(3,3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
model= get_cnn_model()
model.optimizer.lr=0.01

In [ ]:
history=model.fit_generator(generator=batches, steps_per_epoch=X_train.shape[0]//64, epochs=10, 
                    validation_data=val_batches, validation_steps=val_batches.n)

# Data Augmentation

In [ ]:
gen =ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size=64)
val_batches = gen.flow(X_val, y_val, batch_size=64)

In [ ]:
model.optimizer.lr=0.001
history=model.fit_generator(generator=batches, steps_per_epoch=X_train.shape[0]//64, epochs=3, 
                    validation_data=val_batches, validation_steps=val_batches.n)

# Adding Batch Normalization

In [ ]:
from keras.layers import BatchNormalization

def get_bn_model():
    model = Sequential([
        Lambda(standardize,input_shape=(28,28,1)),
        Convolution2D(32,(3,3), activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,(3,3), activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,(3,3), activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,(3,3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512,activation='relu'),
        BatchNormalization(),
        Dense(10,activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model= get_bn_model()
model.optimizer.lr=0.01
history=model.fit_generator(generator=batches, steps_per_epoch=X_train.shape[0]//64, epochs=5, 
                    validation_data=val_batches, validation_steps=val_batches.n)

# Submission to Kaggle

In [ ]:
predictions = model.predict_classes(X_test, verbose=0)
submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("DR.csv", index=False, header=True)

In [ ]:
submissions.head()